# Step 3: Model Operationalization & Deployment

In the `Code\2_model_building_and_evaluation.ipynb` notebook, we trained an LSTM neural net to predict aircraft engine failure 30 cycles into the future.

In this notebook, we will create the artifacts and scripts to deploy the LSTM model into a webservice on Azure. The artifacts include the model files, and test scripts to validate your model can be used to predict future reliability of the engines based on the present operating characteristics.

In [1]:
import keras
# import the libraries
import os
import pandas as pd
import numpy as np
import json
import shutil
from keras.models import model_from_json

import h5py

# For creating the deployment schema file
from azureml.api.schema.dataTypes import DataTypes
from azureml.api.schema.sampleDefinition import SampleDefinition
from azureml.api.realtime.services import generate_schema

# Use the Azure Machine Learning data collector to log various metrics
from azureml.logging import get_azureml_logger
run_logger = get_azureml_logger()
run_logger.log('amlrealworld.predictivemaintenanceforpm.operationalization','true')

# For Azure blob storage access
from azure.storage.blob import BlockBlobService
from azure.storage.blob import PublicAccess

Using TensorFlow backend.


## Model storage 

We will store the model and operationalization artifacts in an Azure Blob Storage Container for easy retrieval to your deployment platform.

Instructions for setting up your Azure Storage account are available within this link (https://docs.microsoft.com/en-us/azure/storage/blobs/storage-python-how-to-use-blob-storage). You will need to copy your account name and account key from the _Access Keys_ area in the portal into the following code block. These credentials will be reused in all four Jupyter notebooks.

We will handle creating the containers and writing the data to these containers for each notebook. Further instructions for using Azure Blob storage with AML Workbench are available (https://github.com/Azure/ViennaDocs/blob/master/Documentation/UsingBlobForStorage.md).

You will need to enter the **ACCOUNT_NAME** as well as the **ACCOUNT_KEY** in order to access Azure Blob storage account you have created. This notebook will create and store all the resulting data files in a blob container under this account.


In [2]:
# Enter your Azure blob storage details here 
ACCOUNT_NAME = "<your blob storage account name>"

# You can find the account key under the _Access Keys_ link in the 
# [Azure Portal](portal.azure.com) page for your Azure storage container.
ACCOUNT_KEY = "<your blob storage account key>"

#-------------------------------------------------------------------------------------------
# The data from the Data Ingestion and Preparation notebook is stored in the sensordata ingestion container.
MODEL_CONTAINER = "pmlstmmodel" 
# Connect to your blob service     
az_blob_service = BlockBlobService(account_name=ACCOUNT_NAME, account_key=ACCOUNT_KEY)

These variables tell the notebook where to store and load files for building and testing the model.

In [3]:
# We will store each of these data sets in a local persistance folder
SHARE_ROOT = os.environ['AZUREML_NATIVE_SHARE_DIRECTORY']

# These file names detail the data files. 
TEST_DATA = 'PM_test_files.pkl'

# We'll serialize the model in json format
LSTM_MODEL = 'modellstm.json'

# and store the weights in h5
MODEL_WEIGHTS = 'modellstm.h5'

# and the schema file
SCHEMA_FILE = 'service_schema.json'

## Load the test data frame

We have previously stored the test data frame in the local persistence directory indicated by the `SHARE_ROOT` variable. We'll use this test data frame to build the model schema to describe the deployment function calls.

In [4]:
test_df = pd.read_pickle(SHARE_ROOT + TEST_DATA)
test_df.head(10)

,id,cycle,setting1,setting2,setting3,s1,s2,s3,s4,s5,...,s16,s17,s18,s19,s20,s21,cycle_norm,RUL,label1,label2
0,1,1,0.632184,0.750000,0.0,0.0,0.545181,0.310661,0.269413,0.0,...,0.0,0.333333,0.0,0.0,0.558140,0.661834,0.000000,142,0,0
1,1,2,0.344828,0.250000,0.0,0.0,0.150602,0.379551,0.222316,0.0,...,0.0,0.416667,0.0,0.0,0.682171,0.686827,0.002770,141,0,0
2,1,3,0.517241,0.583333,0.0,0.0,0.376506,0.346632,0.322248,0.0,...,0.0,0.416667,0.0,0.0,0.728682,0.721348,0.005540,140,0,0
3,1,4,0.741379,0.500000,0.0,0.0,0.370482,0.285154,0.408001,0.0,...,0.0,0.250000,0.0,0.0,0.666667,0.662110,0.008310,139,0,0
4,1,5,0.580460,0.500000,0.0,0.0,0.391566,0.352082,0.332039,0.0,...,0.0,0.166667,0.0,0.0,0.658915,0.716377,0.011080,138,0,0
5,1,6,0.568966,0.750000,0.0,0.0,0.271084,0.176150,0.217421,0.0,...,0.0,0.333333,0.0,0.0,0.596899,0.624827,0.013850,137,0,0
6,1,7,0.500000,0.666667,0.0,0.0,0.271084,0.268149,0.381330,0.0,...,0.0,0.250000,0.0,0.0,0.550388,0.691798,0.016620,136,0,0
7,1,8,0.534483,0.500000,0.0,0.0,0.400602,0.214737,0.314652,0.0,...,0.0,0.416667,0.0,0.0,0.705426,0.591273,0.019391,135,0,0
8,1,9,0.293103,0.500000,0.0,0.0,0.201807,0.485066,0.506921,0.0,...,0.0,0.250000,0.0,0.0,0.744186,0.770367,0.022161,134,0,0
9,1,10,0.356322,0.416667,0.0,0.0,0.259036,0.309789,0.276671,0.0,...,0.0,0.250000,0.0,0.0,0.565891,0.673571,0.024931,133,0,0


We will need to recreate the feature engineering (creating the sequence features) just as we did in the model building step. We will do this within the webservice so that the service can take the raw sensor data, and return a scored result predicting probability of failure at 30 days (`label1`). 

When scoring an unseen observation, the model will not know the true labels. Therefore, we create a `score_df` without labels.

In [5]:
# pick the feature columns 
# Sequence help order the observations in "time"
sequence_cols = ['setting1', 'setting2', 'setting3', 'cycle_norm']

# key columns group the machines
key_cols = ['id', 'cycle']

# Labels are what we're predicting.
label_cols = ['label1', 'label2', 'RUL']

# The scoreing data should not have labels... if we knew the label, 
# we wouldn'y need to predict.
score_df = test_df.drop(label_cols, axis = 1)


### Test init() and run() functions

The web service requires two functions, an `init()` function that will initialize the web service by loading the model into the service, and a `run()` function that will engineer the features to match the model call structure, and score that data set. We create the functions in here for testing and debugging.

In [6]:
def init():
    # read in the model file
    from keras.models import model_from_json
    global loaded_model
    
    # load json and create model
    with open(SHARE_ROOT + LSTM_MODEL, 'r') as json_file:
        loaded_model_json = json_file.read()
        json_file.close()
        loaded_model = model_from_json(loaded_model_json)
    
    # load weights into new model
    loaded_model.load_weights(os.path.join(SHARE_ROOT, MODEL_WEIGHTS))
    loaded_model.compile('sgd','mse')
    

In [7]:
def run(score_input): 
    # Create the sequences
    sequence_length = 50
    sequence_cols = ['setting1', 'setting2', 'setting3', 'cycle_norm']
    key_cols = ['id', 'cycle']

    # Feature engineering
    input_features = score_input.columns.values.tolist()
    sensor_cols = [x for x in input_features if x not in set(key_cols)]
    sensor_cols = [x for x in sensor_cols if x not in set(sequence_cols)]

    # The time is sequenced along
    # This may be a silly way to get these column names, but it's relatively clear
    sequence_cols.extend(sensor_cols)
    
    seq_array = [score_input[score_input['id']==id][sequence_cols].values[-sequence_length:] 
                 for id in score_input['id'].unique() if len(score_input[score_input['id']==id]) >= sequence_length]

    seq_array = np.asarray(seq_array).astype(np.float32)
    try:
        prediction = loaded_model.predict_proba(seq_array)
        #print(prediction)
        pred = prediction.tolist()
        return(pred)
    except Exception as e:
        return(str(e))
    

Check that there are 100 unique engine IDs. 

In [8]:
print(score_df.id.unique())

[  1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18
  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36
  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54
  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72
  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90
  91  92  93  94  95  96  97  98  99 100]


The webservice test requires an `initialize` of the webservice, then send the entire scoring data set into the model. We expect to get 1 probability prediction for each engine in the scoring data set. Since the `score_df` has 100 machines, we expect 100 probabilities back.

In [9]:
init()

prb=run(score_df)
print(prb)

[[0.00375646841712296], [0.00033805289422161877], [0.0007565882406197488], [0.0003128668759018183], [6.648356793448329e-05], [0.0005828844732604921], [0.0002597148122731596], [0.00018564850324764848], [0.00030933888047002256], [7.479621126549318e-05], [0.00015490273653995246], [8.803210948826745e-05], [0.00010222086712019518], [0.0108488779515028], [0.5783206224441528], [0.00016159925144165754], [0.9886316657066345], [0.00016178519581444561], [4.838027962250635e-05], [0.9821566939353943], [9.348896855954081e-05], [0.00038087248685769737], [0.00023769873951096088], [0.0002285395748913288], [0.0005704331560991704], [0.9989494681358337], [0.004981394857168198], [0.0001682192669250071], [0.9991666078567505], [0.9897217154502869], [0.9333099126815796], [0.823441743850708], [0.0020724621135741472], [0.8350445628166199], [0.9192692041397095], [0.9986440539360046], [0.0010718094417825341], [0.0002111715730279684], [0.0007312165107578039], [0.05197881534695625], [0.00028019360615871847], [0.000

Instead we get 93, because 7 of the machines do not have the full 50 cycles available for scoring. If we send a machine with fewer than 50 records we get the following back: 

In [10]:
tst_df=score_df.loc[score_df['id'] == 1]

print(tst_df.shape)

# Because 
run(tst_df)

(31, 27)


'Error when checking : expected lstm_1_input to have 3 dimensions, but got array with shape (0, 1)'

If we send a complete data set, like machineID == 3, we get a probability back.

In [11]:
tst_df=score_df.loc[score_df['id'] == 3]

print(tst_df.shape)

# Because 
ans=run(tst_df)

print(ans)

(126, 27)
[[0.00375646841712296]]


## Persist model assets

Next we persist the assets we have created for use in operationalization. First we need to define the schema so the webservice knows what the payload data will look like as it comes in.

In [12]:
# define the input data frame
inputs = {"score_input": SampleDefinition(DataTypes.PANDAS, score_df)}

json_schema = generate_schema(run_func=run, inputs=inputs, filepath=SCHEMA_FILE)

# save the schema file for deployment
out = json.dumps(json_schema)
with open(SHARE_ROOT + SCHEMA_FILE, 'w') as f:
    f.write(out)

The conda dependencies are defined in this `webservices_conda.yaml` file. This will be used to tell the webservice server which python packages are required to run this web service.

In [13]:
%%writefile {SHARE_ROOT}/webservices_conda.yaml

# Conda environment specification. The dependencies defined in this file will
# be automatically provisioned for managed runs. These include runs against
# the localdocker, remotedocker, and cluster compute targets.

# Note that this file is NOT used to automatically manage dependencies for the
# local compute target. To provision these dependencies locally, run:
# conda env update --file conda_dependencies.yml

# Details about the Conda environment file format:
# https://conda.io/docs/using/envs.html#create-environment-file-by-hand

# For managing Spark packages and configuration, see spark_dependencies.yml.

name: project_environment
channels:
- conda-forge
- defaults
dependencies:
  - python=3.5.2
  - pip:
    - azure-common==1.1.8
    - azure-storage==0.36.0
    - numpy==1.14.0 
    - sklearn
    - keras
    - tensorflow
    - h5py

Writing /azureml-share//webservices_conda.yaml


The `lstmscore.py` file is python code defining the web service operation. It includes both the `init()` and `run()` functions defined earlier imports the required libraries. These should be nearly identical to the previous defined versions.

In [14]:
%%writefile {SHARE_ROOT}/lstmscore.py

# import the libraries
import keras
import tensorflow
import json
import shutil
import numpy as np


def init():
    # read in the model file
    from keras.models import model_from_json
    global loaded_model
    
    # load json and create model
    with open('modellstm.json', 'r') as json_file:
        loaded_model_json = json_file.read()
        json_file.close()
        loaded_model = model_from_json(loaded_model_json)
    
    # load weights into new model
    loaded_model.load_weights("modellstm.h5")
    loaded_model.compile('sgd','mse')

def run(score_input):
    # Create the sequences
    sequence_length = 50
    sequence_cols = ['setting1', 'setting2', 'setting3', 'cycle_norm']
    key_cols = ['id', 'cycle']

    input_features = score_input.columns.values.tolist()
    sensor_cols = [x for x in input_features if x not in set(key_cols)]
    sensor_cols = [x for x in sensor_cols if x not in set(sequence_cols)]

    # The time is sequenced along
    # This may be a silly way to get these column names, but it's relatively clear
    sequence_cols.extend(sensor_cols)
    
    seq_array = [score_input[score_input['id']==id][sequence_cols].values[-sequence_length:] 
                 for id in score_input['id'].unique() if len(score_input[score_input['id']==id]) >= sequence_length]

    seq_array = np.asarray(seq_array).astype(np.float32)
    try:
        prediction = loaded_model.predict_proba(seq_array)
        
        pred = prediction.tolist()
        return(pred)
    except Exception as e:
        return(str(e))
    
if __name__ == "__main__":
    init()
    run("{\"score_df\": [{\"s20\": 0.5581395348837184, \"s10\": 0.0, \"s2\": 0.5451807228915584, \"s21\": 0.6618337475835432, \"s9\": 0.12761374854168395, \"s19\": 0.0, \"s3\": 0.31066056245912677, \"cycle\": 1, \"s15\": 0.3089649865332831, \"s4\": 0.2694125590817009, \"s1\": 0.0, \"s11\": 0.2083333333333357, \"cycle_norm\": 0.0, \"s13\": 0.2205882352941444, \"s5\": 0.0, \"s18\": 0.0, \"s8\": 0.2121212121210192, \"s14\": 0.1321601816492901, \"s6\": 1.0, \"setting2\": 0.75, \"setting1\": 0.632183908045977, \"s12\": 0.6460554371002161, \"s17\": 0.3333333333333357, \"s16\": 0.0, \"id\": 1, \"setting3\": 0.0, \"s7\": 0.6521739130434696}, {\"s20\": 0.6821705426356601, \"s10\": 0.0, \"s2\": 0.15060240963856586, \"s21\": 0.6868268434134208, \"s9\": 0.14668401687158195, \"s19\": 0.0, \"s3\": 0.37955090473076325, \"cycle\": 2, \"s15\": 0.21315890727203168, \"s4\": 0.2223160027008788, \"s1\": 0.0, \"s11\": 0.38690476190476275, \"cycle_norm\": 0.002770083102493075, \"s13\": 0.26470588235270043, \"s5\": 0.0, \"s18\": 0.0, \"s8\": 0.16666666666696983, \"s14\": 0.20476829394158358, \"s6\": 1.0, \"setting2\": 0.25, \"setting1\": 0.3448275862068965, \"s12\": 0.7398720682302695, \"s17\": 0.4166666666666714, \"s16\": 0.0, \"id\": 1, \"setting3\": 0.0, \"s7\": 0.8051529790660226}, {\"s20\": 0.7286821705426334, \"s10\": 0.0, \"s2\": 0.3765060240963862, \"s21\": 0.7213476940071786, \"s9\": 0.15808130664991182, \"s19\": 0.0, \"s3\": 0.34663178548071016, \"cycle\": 3, \"s15\": 0.4586379376683354, \"s4\": 0.3222484807562438, \"s1\": 0.0, \"s11\": 0.38690476190476275, \"cycle_norm\": 0.0055401662049861505, \"s13\": 0.2205882352941444, \"s5\": 0.0, \"s18\": 0.0, \"s8\": 0.22727272727297532, \"s14\": 0.15564041696769948, \"s6\": 1.0, \"setting2\": 0.5833333333333334, \"setting1\": 0.5172413793103449, \"s12\": 0.6993603411513902, \"s17\": 0.4166666666666714, \"s16\": 0.0, \"id\": 1, \"setting3\": 0.0, \"s7\": 0.6859903381642596}]}")

Overwriting /azureml-share//lstmscore.py


We also include a python file `test_service.py` which can test the web service you create. This program will test that there are enough cycles for the webservice to score, and send a single engine set to the web service. The results are printed with the engine ID to simulate how to use the webservice in a production setting.

After you create the web service, you will need to edit this file and provide the correct `url` endpoint string before running this program.  

In [15]:
%%writefile {SHARE_ROOT}/test_service.py

import urllib
import json 
import requests
import pandas as pd

# The URL will need to be editted after service create.
url = 'http://127.0.0.1:32773/score'

## Sequence length will need to match the training sequence length from
## 2_model_building_and_evaluation.ipynb
sequence_length = 50

# We'll read in this data to test the service
test_df = pd.read_pickle('PM_test_files.pkl')

# Labels are what we're predicting.
label_cols = ['label1', 'label2', 'RUL']

# The scoreing data should not have labels... if we knew the label, 
# we wouldn't need to predict.
score_df = test_df.drop(label_cols, axis = 1)
headers = {'Content-Type':'application/json'}

# Now get the machine numbers, for each machine get the 
# prediction for the label timepoint
machineID = score_df['id'].unique()

for ind in machineID:
    
    try:
        body = score_df[score_df.id==ind]
        print('ID {}: size {}'.format(ind, body.shape))
        if body.shape[0] < sequence_length : 
            print("Skipping machineID {} as we need {} records to score and only have {} records.".format(ind, sequence_length, body.shape[0]))
            continue
        print('ID {}: {} \t {}'.format(ind, body.shape, body.tail(sequence_length+ 10).shape))
        body = "{\"score_input\": " + \
                    body.tail(sequence_length+10).to_json(orient="records") +\
                    "}"
        
        req = urllib.request.Request(url, str.encode(body), headers) 
        with urllib.request.urlopen(req) as response:
            the_page = response.read()
            print('ID {}: {}'.format(ind,the_page))
        
    except urllib.error.HTTPError as error:
        print("The request failed with status code {}: \n{}".format(error, error.read))

        # Print the headers - they include the requert ID and the timestamp, which are useful for debugging the failure
        print(error.info())
        print(error.reason)      

Writing /azureml-share//test_service.py


## Packaging

To move the model artifacts around, we'll zip them into one file. We can then retrieve this file from the persistence shared folder on your DSVM.

https://docs.microsoft.com/en-us/azure/machine-learning/preview/how-to-read-write-files



In [16]:
# Compress the operationalization assets for easy blob storage transfer
# We can remove the persisted data files.
# !rm {SHARE_ROOT}/PM*.pkl
!rm {SHARE_ROOT}/PM_train_files.pkl
!ls {SHARE_ROOT}

MODEL_O16N = shutil.make_archive('LSTM_o16n', 'zip', SHARE_ROOT)

# Create a new container if necessary, otherwise you can use an existing container.
# This command creates the container if it does not already exist. Else it does nothing.
az_blob_service.create_container(MODEL_CONTAINER,
                                 fail_on_exist=False, 
                                 public_access=PublicAccess.Container)

# Transfer the compressed operationalization assets into the blob container.
az_blob_service.create_blob_from_path(MODEL_CONTAINER, "LSTM_o16n.zip", str(MODEL_O16N)) 

PM_test_files.pkl  modellstm.h5    service_schema.json	webservices_conda.yaml
lstmscore.py	   modellstm.json  test_service.py


# Deployment

Once the assets are stored, we can download them into a deployment compute context for operationalization on an Azure web service. For this scenario, we will deploy this on our local docker container context.

We demonstrate how to setup this web service this through a CLI window opened in the AML Workbench application. 

## Download the model

To download the model we've saved, follow these instructions on a local computer.

 - Open the Azure Portal
 - In the left hand pane, click on All resources
 - Search for the storage account using the name you provided earlier in this notebook.
 - Choose the storage account from search result list, this will open the storage account panel.
 - On the storage account panel, choose Blobs
 - On the Blobs panel choose the container `pmlstmmodel`
 - Select the file `LSTM_o16n.zip` and on the properties pane for that blob, choose download.

Once downloaded, unzip the file into the directory of your choosing. The zip file contains the deployment assets:

- the `lstmscore.py` file which contains functionst to do the model scoring
- the `modellstm.json` model definition file
- the `modellstm.h5` model weights file
- the `service_schema.json` which defines the input data schema
- the `webservices_conda.yaml` defining which python packages are required to run this service.
- the `test_service.py` script for testing your webservice once deployed.
- the `PM_test_df.pkl` test data set used by the `test_service.py` script


## Create a model management endpoint 

Create a modelmanagement under your account. We will call this `pdmmodelmanagement`. You only need to do this once for any AMLWB models you would like to deploy. You will need to supply an **ACCOUNT_REGION** and **RESOURCE_GROUP** from your Azure account. The remaining defaults are acceptable.

`az ml account modelmanagement create --location <ACCOUNT_REGION> --resource-group <RESOURCE_GROUP> --name pdmmodelmanagement`

You can find the correct text for the `<ACCOUNT_REGION>` using the command:

`az account list-locations`

If you get a `ResourceGroupNotFound` error, you may need to set the correct subscription. This is typically only an issue if your Azure login connects to multiple subscritpions. 

`az account set -s '<subscription name>'`

You can find the `subscription name` or `subscription id` through the (https://portal.azure.com) under the resource group you'd like to use.

## Check environment settings

Show what environment is currently active:

`az ml env show`

If nothing is set, we setup the environment with an existing model management context (created above) first: 

` az ml env setup --location <ACCOUNT_REGION> --resource-group <RESOURCE_GROUP> --name pdmmodelmanagement`

using the same `<ACCOUNT_REGION>` and `<RESOURCE_GROUP>` in the previous section. Then set the current environment:

`az ml env set --resource-group <RESOURCE_GROUP> --cluster-name pdmmodelmanagement`

Check that the environment is now set:

`az ml env show`

## Deploy a web service 

These commands assume the current directory contains the webservice assets we created in throughout the notebooks in this scenario (at least `lstmscore.py`, `modellstm.json`, `modellstm.h5`, `service_schema.json` and `webservices_conda.yaml`). If not, in the AML CLI window, change to the directory where the zip file was unpacked. 

The command to create a web service (`<SERVICE_ID>`) with these operationalization assets in the current directory is:

`
az ml service create realtime -f <filename> -r <TARGET_RUNTIME> -m <MODEL_FILE> -s <SCHEMA_FILE> -n <SERVICE_ID> --cpu 0.1
`

The default cluster has only 2 nodes with 2 cores each. Some cores are taken for system components. AMLWorkbench asks for 1 core per service. To deploy multiple services into this cluster, we specify the cpu requirement in the service create command as (--cpu 0.1) to request 10% of a core. 

For this example, we will call our webservice with the `SERVICE_ID` = `lstmwebservice`. The `SERVICE_ID` must be all lowercase, with no spaces. This command should work with your account and the deployment artifacts created in this notebook.

`
az ml service create realtime -f lstmscore.py -r python -m modellstm.json -m modellstm.h5 -s service_schema.json -c webservices_conda.yaml --cpu 0.1 -n lstmwebservice
`

This command will take some time to execute. 

## Test your deployment.

Once complete, the `az ml service create` command returns sample usage commands to test the service for both PowerShell and the cmd prompt. You can copy and paste this command into the CLI to test the web service. However, since it is only an example with 1 cycle, we know the model can not return a reasonable score. Instead you will see the following error as detailed above:
```
'Error when checking : expected lstm_1_input to have 3 dimensions, but got array with shape (0, 1)'

```
We have provided the `test_service.py` python script to send larger payloads to the service. First obtain the webservice endpoint with the following command. `az ml service usage realtime -i lstmwebservice`

```
> az ml service usage realtime -i lstmwebservice
Scoring URL:
    http://127.0.0.1:32770/score

Headers:
    Content-Type: application/json

Swagger URL:
    http://127.0.0.1:32770/swagger.json

Sample CLI command:
...
```

Copy the Scoring URL into the `url` variable in the `test_service.py` script and save the file.

From the CLI, you can then run the script with the command and response similar to the following:

`python test_service.py`

```
ID 1: size (31, 27)
Skipping machineID 1 as we need 50 records to score and only have 31 records.
ID 2: size (49, 27)
Skipping machineID 2 as we need 50 records to score and only have 49 records.
ID 3: size (126, 27)
ID 3: (126, 27)          (60, 27)
ID 3: b'"[[0.003690145444124937]]"'
ID 4: size (106, 27)
ID 4: (106, 27)          (60, 27)
ID 4: b'"[[0.0014941827394068241]]"'
ID 5: size (98, 27)
ID 5: (98, 27)   (60, 27)
ID 5: b'"[[0.002231441903859377]]"'
ID 6: size (105, 27)
ID 6: (105, 27)          (60, 27)
ID 6: b'"[[0.001609073020517826]]"'
ID 7: size (160, 27)
ID 7: (160, 27)          (60, 27)
ID 7: b'"[[0.001093236613087356]]"'
ID 8: size (166, 27)
ID 8: (166, 27)          (60, 27)
ID 8: b'"[[0.0017530957702547312]]"'
ID 9: size (55, 27)
ID 9: (55, 27)   (55, 27)
ID 9: b'"[[0.0015462590381503105]]"'
ID 10: size (192, 27)
ID 10: (192, 27)         (60, 27)
ID 10: b'"[[0.0014192759990692139]]"'
ID 11: size (83, 27)
ID 11: (83, 27)          (60, 27)
ID 11: b'"[[0.0017256727442145348]]"'
ID 12: size (217, 27)
ID 12: (217, 27)         (60, 27)
ID 12: b'"[[0.0011249856324866414]]"'
ID 13: size (195, 27)
ID 13: (195, 27)         (60, 27)
ID 13: b'"[[0.0013882251223549247]]"'
ID 14: size (46, 27)
Skipping machineID 14 as we need 50 records to score and only have 46 records.
ID 15: size (76, 27)
ID 15: (76, 27)          (60, 27)
ID 15: b'"[[0.0010017495369538665]]"'
ID 16: size (113, 27)
ID 16: (113, 27)         (60, 27)
ID 16: b'"[[0.0011311823036521673]]"'
ID 17: size (165, 27)
ID 17: (165, 27)         (60, 27)
ID 17: b'"[[0.004585478454828262]]"'
ID 18: size (133, 27)
ID 18: (133, 27)         (60, 27)
ID 18: b'"[[0.08827716112136841]]"'
ID 19: size (135, 27)
ID 19: (135, 27)         (60, 27)
ID 19: b'"[[0.0012705528642982244]]"'
ID 20: size (184, 27)
ID 20: (184, 27)         (60, 27)
ID 20: b'"[[0.531469464302063]]"'
ID 21: size (148, 27)
ID 21: (148, 27)         (60, 27)
ID 21: b'"[[0.0013451582053676248]]"'
ID 22: size (39, 27)
Skipping machineID 22 as we need 50 records to score and only have 39 records.
ID 23: size (130, 27)
ID 23: (130, 27)         (60, 27)
ID 23: b'"[[0.0008326334645971656]]"'
ID 24: size (186, 27)
ID 24: (186, 27)         (60, 27)
ID 24: b'"[[0.8084099888801575]]"'
ID 25: size (48, 27)
Skipping machineID 25 as we need 50 records to score and only have 48 records.
ID 26: size (76, 27)
ID 26: (76, 27)          (60, 27)
ID 26: b'"[[0.0009542660554870963]]"'
ID 27: size (140, 27)
ID 27: (140, 27)         (60, 27)
ID 27: b'"[[0.0021106707863509655]]"'
ID 28: size (158, 27)
ID 28: (158, 27)         (60, 27)
ID 28: b'"[[0.0013031685957685113]]"'
ID 29: size (171, 27)
ID 29: (171, 27)         (60, 27)
ID 29: b'"[[0.0016013996209949255]]"'
ID 30: size (143, 27)
ID 30: (143, 27)         (60, 27)
ID 30: b'"[[0.001739262486808002]]"'
ID 31: size (196, 27)
ID 31: (196, 27)         (60, 27)
ID 31: b'"[[0.9811599850654602]]"'
ID 32: size (145, 27)
ID 32: (145, 27)         (60, 27)
ID 32: b'"[[0.004659396130591631]]"'
ID 33: size (50, 27)
ID 33: (50, 27)          (50, 27)
ID 33: b'"[[0.0014037484070286155]]"'
ID 34: size (203, 27)
ID 34: (203, 27)         (60, 27)
ID 34: b'"[[0.9670061469078064]]"'
ID 35: size (198, 27)
ID 35: (198, 27)         (60, 27)
ID 35: b'"[[0.9815614819526672]]"'
ID 36: size (126, 27)
ID 36: (126, 27)         (60, 27)
ID 36: b'"[[0.6011714339256287]]"'
ID 37: size (121, 27)
ID 37: (121, 27)         (60, 27)
ID 37: b'"[[0.09062018990516663]]"'
ID 38: size (125, 27)
ID 38: (125, 27)         (60, 27)
ID 38: b'"[[0.004488768056035042]]"'
ID 39: size (37, 27)
Skipping machineID 39 as we need 50 records to score and only have 37 records.
ID 40: size (133, 27)
ID 40: (133, 27)         (60, 27)
ID 40: b'"[[0.2266664057970047]]"'
ID 41: size (123, 27)
ID 41: (123, 27)         (60, 27)
ID 41: b'"[[0.34560075402259827]]"'
ID 42: size (156, 27)
ID 42: (156, 27)         (60, 27)
ID 42: b'"[[0.9412360191345215]]"'
ID 43: size (172, 27)
ID 43: (172, 27)         (60, 27)
ID 43: b'"[[0.0023652282543480396]]"'
ID 44: size (54, 27)
ID 44: (54, 27)          (54, 27)
ID 44: b'"[[0.0016468992689624429]]"'
ID 45: size (152, 27)
ID 45: (152, 27)         (60, 27)
ID 45: b'"[[0.0028006499633193016]]"'
ID 46: size (146, 27)
ID 46: (146, 27)         (60, 27)
ID 46: b'"[[0.01627732440829277]]"'
ID 47: size (73, 27)
ID 47: (73, 27)          (60, 27)
ID 47: b'"[[0.001768825575709343]]"'
ID 48: size (78, 27)
ID 48: (78, 27)          (60, 27)
ID 48: b'"[[0.0014678625157102942]]"'
ID 49: size (303, 27)
ID 49: (303, 27)         (60, 27)
ID 49: b'"[[0.20870330929756165]]"'
ID 50: size (74, 27)
ID 50: (74, 27)          (60, 27)
ID 50: b'"[[0.0012800053227692842]]"'
ID 51: size (144, 27)
ID 51: (144, 27)         (60, 27)
ID 51: b'"[[0.0018505703192204237]]"'
ID 52: size (189, 27)
ID 52: (189, 27)         (60, 27)
ID 52: b'"[[0.05433798208832741]]"'
ID 53: size (164, 27)
ID 53: (164, 27)         (60, 27)
ID 53: b'"[[0.043289054185152054]]"'
ID 54: size (121, 27)
ID 54: (121, 27)         (60, 27)
ID 54: b'"[[0.0008320639608427882]]"'
ID 55: size (113, 27)
ID 55: (113, 27)         (60, 27)
ID 55: b'"[[0.0010971747105941176]]"'
ID 56: size (136, 27)
ID 56: (136, 27)         (60, 27)
ID 56: b'"[[0.935263991355896]]"'
ID 57: size (160, 27)
ID 57: (160, 27)         (60, 27)
ID 57: b'"[[0.001842512865550816]]"'
ID 58: size (176, 27)
ID 58: (176, 27)         (60, 27)
ID 58: b'"[[0.008606999181210995]]"'
ID 59: size (94, 27)
ID 59: (94, 27)          (60, 27)
ID 59: b'"[[0.000816409767139703]]"'
ID 60: size (147, 27)
ID 60: (147, 27)         (60, 27)
ID 60: b'"[[0.0018146623624488711]]"'
ID 61: size (159, 27)
ID 61: (159, 27)         (60, 27)
ID 61: b'"[[0.16924026608467102]]"'
ID 62: size (232, 27)
ID 62: (232, 27)         (60, 27)
ID 62: b'"[[0.006756726652383804]]"'
ID 63: size (155, 27)
ID 63: (155, 27)         (60, 27)
ID 63: b'"[[0.004599822219461203]]"'
ID 64: size (168, 27)
ID 64: (168, 27)         (60, 27)
ID 64: b'"[[0.1045515388250351]]"'
ID 65: size (71, 27)
ID 65: (71, 27)          (60, 27)
ID 65: b'"[[0.0006783730932511389]]"'
ID 66: size (147, 27)
ID 66: (147, 27)         (60, 27)
ID 66: b'"[[0.5332847237586975]]"'
ID 67: size (71, 27)
ID 67: (71, 27)          (60, 27)
ID 67: b'"[[0.0010836098808795214]]"'
ID 68: size (187, 27)
ID 68: (187, 27)         (60, 27)
ID 68: b'"[[0.9780803918838501]]"'
ID 69: size (54, 27)
ID 69: (54, 27)          (54, 27)
ID 69: b'"[[0.0013221375411376357]]"'
ID 70: size (152, 27)
ID 70: (152, 27)         (60, 27)
ID 70: b'"[[0.0018269383581355214]]"'
ID 71: size (68, 27)
ID 71: (68, 27)          (60, 27)
ID 71: b'"[[0.0011093434877693653]]"'
ID 72: size (131, 27)
ID 72: (131, 27)         (60, 27)
ID 72: b'"[[0.0025075972080230713]]"'
ID 73: size (112, 27)
ID 73: (112, 27)         (60, 27)
ID 73: b'"[[0.0010514295427128673]]"'
ID 74: size (137, 27)
ID 74: (137, 27)         (60, 27)
ID 74: b'"[[0.001497327582910657]]"'
ID 75: size (88, 27)
ID 75: (88, 27)          (60, 27)
ID 75: b'"[[0.0013855294091627002]]"'
ID 76: size (205, 27)
ID 76: (205, 27)         (60, 27)
ID 76: b'"[[0.9812294244766235]]"'
ID 77: size (162, 27)
ID 77: (162, 27)         (60, 27)
ID 77: b'"[[0.03927205502986908]]"'
ID 78: size (72, 27)
ID 78: (72, 27)          (60, 27)
ID 78: b'"[[0.0005832962924614549]]"'
ID 79: size (101, 27)
ID 79: (101, 27)         (60, 27)
ID 79: b'"[[0.0019143277313560247]]"'
ID 80: size (133, 27)
ID 80: (133, 27)         (60, 27)
ID 80: b'"[[0.0017474571941420436]]"'
ID 81: size (213, 27)
ID 81: (213, 27)         (60, 27)
ID 81: b'"[[0.9732806086540222]]"'
ID 82: size (162, 27)
ID 82: (162, 27)         (60, 27)
ID 82: b'"[[0.9421394467353821]]"'
ID 83: size (73, 27)
ID 83: (73, 27)          (60, 27)
ID 83: b'"[[0.0011075431248173118]]"'
ID 84: size (172, 27)
ID 84: (172, 27)         (60, 27)
ID 84: b'"[[0.0032924318220466375]]"'
ID 85: size (34, 27)
Skipping machineID 85 as we need 50 records to score and only have 34 records.
ID 86: size (110, 27)
ID 86: (110, 27)         (60, 27)
ID 86: b'"[[0.001197033328935504]]"'
ID 87: size (56, 27)
ID 87: (56, 27)          (56, 27)
ID 87: b'"[[0.0008590950164943933]]"'
ID 88: size (68, 27)
ID 88: (68, 27)          (60, 27)
ID 88: b'"[[0.001377861830405891]]"'
ID 89: size (177, 27)
ID 89: (177, 27)         (60, 27)
ID 89: b'"[[0.0012676144251599908]]"'
ID 90: size (146, 27)
ID 90: (146, 27)         (60, 27)
ID 90: b'"[[0.13571304082870483]]"'
ID 91: size (234, 27)
ID 91: (234, 27)         (60, 27)
ID 91: b'"[[0.2942765951156616]]"'
ID 92: size (150, 27)
ID 92: (150, 27)         (60, 27)
ID 92: b'"[[0.3088526129722595]]"'
ID 93: size (244, 27)
ID 93: (244, 27)         (60, 27)
ID 93: b'"[[0.0115140900015831]]"'
ID 94: size (133, 27)
ID 94: (133, 27)         (60, 27)
ID 94: b'"[[0.004197425674647093]]"'
ID 95: size (89, 27)
ID 95: (89, 27)          (60, 27)
ID 95: b'"[[0.0012697000056505203]]"'
ID 96: size (97, 27)
ID 96: (97, 27)          (60, 27)
ID 96: b'"[[0.0008713220595382154]]"'
ID 97: size (134, 27)
ID 97: (134, 27)         (60, 27)
ID 97: b'"[[0.001297763898037374]]"'
ID 98: size (121, 27)
ID 98: (121, 27)         (60, 27)
ID 98: b'"[[0.0026060494128614664]]"'
ID 99: size (97, 27)
ID 99: (97, 27)          (60, 27)
ID 99: b'"[[0.0007261313730850816]]"'
ID 100: size (198, 27)
ID 100: (198, 27)        (60, 27)
ID 100: b'"[[0.2326587438583374]]"'
```



# Conclusion

Working through all of these notebooks, we have completed:

 * Data aquisition ingestion and preparation in `Code/1_data_ingestion_and_preparation.ipynb` notebook.
 * Building an LSTM time series model that used a 50 cycle history from an aircraft engine to predict failure within the next 30 cycles in the `Code/2_model_building_and_evaluation.ipynb` notebook.
 * Operationalization asset generation and model deployment in the `Code/3_operationalization.ipynb` notebook.
 
This scenario is intended to help guide you through the predictive maintenance model development process with your own data. 

    
 